In [ ]:
import numpy as np
import pyfar as pf
import spharpy
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

import spharpy.beamforming

In [ ]:
def plot_beampattern_azimuth(beampattern, angle, label='Beampattern', colorbar=True, **kwargs):
    fig = plt.gcf()
    axs = plt.gca()
    phase = np.angle(beampattern)
    x = angle
    y = 20*np.log10(np.abs(beampattern))
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    norm = plt.Normalize(0, 2*np.pi)
    lc = LineCollection(segments, cmap=spharpy.plot.phase_twilight(), norm=norm, label=label, **kwargs)
    lc.set_array(phase)
    line = axs.add_collection(lc)
    if colorbar:
        fig.colorbar(line, ax=axs, label='Phase [rad]')


    plt.grid(True)
    ax = plt.gca()
    ax.set_xticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi])
    ax.set_xticklabels(['$0$', '$\pi/2$', '$\pi$',  '$3\pi/2$',  '$2\pi$'])
    ax.set_ylim((-60, 5))
    ax.set_ylabel('Magnitude [dB re 1]')
    ax.set_xlabel('Azimuth [$^\circ$]')
    plt.legend()

# Analytic Plane Wave

In [ ]:
N = 7
doa = pf.Coordinates(-1,0,0)
p_nm = spharpy.spherical.spherical_harmonic_basis_real(N, doa)

# Beamforming

In [ ]:
n_dirs = 1024
azi = np.linspace(0, 2*np.pi, n_dirs)
steering_directions = pf.Coordinates.from_spherical_colatitude(azi, np.ones(n_dirs)*np.pi/2, 1)
Y_steering = spharpy.spherical.spherical_harmonic_basis_real(N, steering_directions)

## Plane Wave Decomposition

In [ ]:
c_nm = 4*np.pi / (N+1)**2

In [ ]:
plane_wave_dencomposition = c_nm * np.squeeze((Y_steering @ p_nm.T))

In [ ]:
plt.figure(figsize=(12, 4))
plot_beampattern_azimuth(plane_wave_dencomposition, azi, label=f'N = {N}')

## Dolph Chebyshev Beamformer

The Dolph-Chebyshev design process consists of either choosing a desired side-lobe attenuation $R$ or main-lobe width $\theta_0$. 

In [ ]:
R_dB = 50
R = 10**(R_dB/20)
d_nm_sidelobe = spharpy.beamforming.dolph_chebyshev_weights(
    N, R, design_criterion='sidelobe')
theta0 = np.pi/6
d_nm_mainlobe = spharpy.beamforming.dolph_chebyshev_weights(
    N, theta0, design_criterion='mainlobe')

In [ ]:
dolph_cheby_sidelobe = np.squeeze(np.real(Y_steering @ np.diag(d_nm_sidelobe) @ p_nm.T))
dolph_cheby_mainlobe = np.squeeze(np.real(Y_steering @ np.diag(d_nm_mainlobe) @ p_nm.T))

In [ ]:
plt.figure(figsize=(12, 4))
plot_beampattern_azimuth(
    dolph_cheby_sidelobe, azi, label=f'N = {N}, R = {R_dB} dB')
plot_beampattern_azimuth(
    dolph_cheby_mainlobe, azi, label='N = {}, $\\theta_0$ = {} $\\pi$'.format(N, np.round(theta0/np.pi, decimals=3)), linestyle='dotted', colorbar=False)
plt.axhline(-R_dB, color='k', linestyle='-.', alpha=0.7)
plt.axvline(theta0+doa.azimuth, color='k', linestyle='-.', alpha=0.7)
plt.axvline(doa.azimuth-theta0, color='k', linestyle='-.', alpha=0.7)

## rE max Weights

In [ ]:
g_nm = spharpy.beamforming.rE_max_weights(N)
re_max = np.squeeze(Y_steering @ np.diag(g_nm) @ p_nm.T)

In [ ]:
plt.figure(figsize=(12, 4))
plot_beampattern_azimuth(re_max, azi, label=f'N = {N}')

## Hanning Window Weighting

In [ ]:
from scipy.signal.windows import hann

In [ ]:
h_n = spharpy.beamforming.normalize_beamforming_weights(
    hann(2*(N+1)+1)[N+1:-1], N)
h_nm = spharpy.indexing.sph_identity_matrix(N).T @ h_n
hanning = np.squeeze(Y_steering @ np.diag(h_nm) @ p_nm.T)

In [ ]:
plt.figure(figsize=(12, 4))
plot_beampattern_azimuth(hanning, azi, label=f'N = {N}')

## Maximum Front-Back Ratio

In [ ]:
f_nm = spharpy.beamforming.maximum_front_back_ratio_weights(N)

In [ ]:
max_fb = np.squeeze(Y_steering @ np.diag(f_nm) @ p_nm.T)

In [ ]:
plt.figure(figsize=(12, 4))
plot_beampattern_azimuth(max_fb, azi, label=f'N = {N}')
plt.axvline(doa.azimuth+np.pi/2, color='k', linestyle='-.', alpha=0.7)
plt.axvline(doa.azimuth-np.pi/2, color='k', linestyle='-.', alpha=0.7)